## Applying the Classifier

Let's apply the classifier to the data we tried to manaully code

In [ ]:
import pandas as pd

pd.set_option('display.max_colwidth', 200)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 300)

%matplotlib inline

## Reading the data

In [ ]:
X_train = pd.read_csv('assaults_downgraded_train.csv', index_col=0)
X_test_with_answers = pd.read_csv('assaults_downgraded_test_with_answers.csv', index_col=0)
X_test = pd.read_csv('assaults_downgraded_test.csv', index_col=0).drop(columns='downgraded').rename(columns={'serious': 'serious_you'})
X_test

## Vectorize & Classify

Vectorize and classify in one big cell!

In [ ]:
from nltk.stem import SnowballStemmer
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_validate

nltk.download('omw-1.4')

# Define stemmer function
stemmer = SnowballStemmer('english')
class StemmedTfidfVectorizer(TfidfVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedTfidfVectorizer,self).build_analyzer()
        return lambda doc:(stemmer.stem(word) for word in analyzer(doc))
    
# vectorize from training set    
vectorizer = StemmedTfidfVectorizer(min_df=15, max_df=0.5)
X = vectorizer.fit_transform(X_train.DO_NARRATIVE)

# classify
y = X_train.serious
clf = LinearSVC()
clf.fit(X, y)

# get scores - cross validate
scores = cross_validate(clf, X, y, cv=10,
                        scoring=('accuracy', 'precision', 'recall', 'f1'))

# here are some other types of scores
# https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
scores_df = pd.DataFrame(scores)
display(scores_df.round(2))
pd.DataFrame(scores)[
    ['fit_time', 'score_time', 'test_accuracy','test_precision','test_recall','test_f1']]\
    .mean().round(2)

## Making predictions

No matter what the terms are that point to a report being filed as Part I or Part II, at the end of the day we're interested in seeing **how good is our model as making predictions?** To test it out, we'll need to perform some predictions on content we know the answer to. Let's start by seeing how it does on some sample sentences.

In [ ]:
X_test_vectors = vectorizer.transform(X_test.DO_NARRATIVE)

In [ ]:
predictions = clf.predict(X_test_vectors)
predictions

In this case `1` means that yes, it was a serious assault. Let's try a few more.

You can see that offenses that include weapons tend to be predicted as serious offenses, while ones involving punching or other direct physical contact are classified as simple assault.

Instead of just looking at which category a report was put in, we can also look at **the score the classifier used for the prediction.**

In [ ]:
prediction_score = clf.decision_function(X_test_vectors)
prediction_score

In [ ]:
X_test_with_answers

In [ ]:
compare_df = X_test.merge(X_test_with_answers[['serious', 'downgraded']], left_index=True, right_index=True)
compare_df['serious_clf'] = predictions
compare_df['serious_clf_pct'] = prediction_score.round(2)
compare_df = compare_df.sort_values(by='serious_clf_pct', ascending=True)
compare_df


# Discussion

What did you get wrong? What did the machine get wrong?

- What are precision errors?
- What are recall errors?

filter compare_df in the cells below...or save it to a CSV and open it up to answer the discussion questions

In [ ]:
compare_df.to_csv('comapre_df.csv')